In [36]:
import random;
import ctypes;
class Array:
    def __init__(self, size):
        assert size > 0
        self.size = size
        PyArrayTypes = ctypes.py_object * size
        self._elements = PyArrayTypes()
        self.clear(None)
        
    def __len__(self):
         return self.size
        
    def __getitem__ (self, index):
        assert index >= 0 and index < self.size
        return self._elements[index]
        
    def __setitem__ (self, index, value):
        assert index >=0 and index < self.size
        self._elements[index] = value 
        
    def clear(self, value):
        for i in range(self.size):
            self._elements[i] = value
                
    def __iter__ (self):
        return _ArrayIterator( self._elements )
    
    def __str__ (self):
        s = "["
        for x in range(self.length()):
            if x != self.length() -1:
                s+= str(self.__getitem__(x)) +','
            else:
                s+= str(self.__getitem__(x))
        s += "]"
        return s
            
        
class _ArrayIterator:
    def __init__(self, thearray):
        self._arrayref = thearray
        self._curNdx = 0;
    
    def __iter__ (self):
        return _ArrayIterator(array._elements)
    
    def __next__(self):
        if self._curNdx < len(self._arrayref):
            entry = self._arrayref[self._curNdx]
            self._curNdx += 1
            return entry
        else:
            raise StopIteration
            
class SparseNode:
    def __init__ (self, value, column):
        self.column = column;
        self.value = value;
        self.next = None;
        
class LinkedList:
    def __init__(self):
        self.head = SparseNode(0, 0);
        
    def add(self, value, column):
        newnode = Node(value, column);
        currentnode = self.head;
        if currentnode is not None:
            while(currentnode.next is not None):
                currentnode = currentnode.next;
            currentnode.next = newnode;
        else:
            self.head = newnode;

class SparseMatrix:
    def __init__(self, row, col):
        self.col = col;
        self.rowlist = Array(row);
            
    def numRows(self):
        return len(self.rowlist);
    
    def numCols(self):
        return self.col;
            
    def getItem(self, indexlist):
        row = indexlist[0];
        col = indexlist[1];
        prevnode = None;
        currentnode = self.rowlist[row];
        while (currentnode is not None and currentnode.column != col):
            prevnode = currentnode;
            currentnode = currentnode.next;
        if currentnode is not None and currentnode.column == col:
            return currentnode.value;
        return 0;
    
    def setItem(self, indexlist, value):
        row = indexlist[0];
        col = indexlist[1];
        newnode = SparseNode(value, col);
        prevnode = None;
        currentnode = self.rowlist[row];
        while(currentnode is not None and col != currentnode.column):
            prevnode = currentnode;
            currentnode = currentnode.next;
        if currentnode is not None and currentnode.column == col:
            if value == 0:
                if currentnode == self.rowlist[row]:
                    self.rowlist[row] = currentnode.next;
                else:
                    prevnode.next = currentnode.next;
            else:
                currentnode.value = value;
            
        elif (value != 0):
            newnode.next = currentnode;
            if currentnode == self.rowlist[row]:
                self.rowlist[row] = newnode;
            else:
                prevnode.next = newnode;
        
    def scaleBy(self, scale):
        for i in range(len(self.rowlist)):
            currentnode = self.rowlist[i];
            if currentnode is not None:
                while (currentnode.next is not None):
                    currentnode.value = currentnode.value * scale;
                    currentnode = currenode.next;
                currentnode.value = currentnode.value * scale
            
        
    def __add__(self, othermatrix):
        assert self.numRows() == othermatrix.numRows();
        assert self.numCols() == othermatrix.numCols();
        newmatrix = SparseMatrix(self.numRows(), self.numCols());
        
        for i in range(self.numRows()):
            currentnode = self.rowlist[i];
            while (currentnode is not None):
                newmatrix.setItem([i, currentnode.column], currentnode.value);
                currentnode = currentnode.next;
                
        for j in range(othermatrix.numRows()):
            currentnode = othermatrix.rowlist[j];
            while (currentnode is not None):
                value = currentnode.value + newmatrix.getItem([j, currentnode.column]);
                newmatrix.setItem([j, currentnode.column], value);
                currentnode = currentnode.next;
        return newmatrix
        
            
    
    def __sub__(self, othermatrix):
        assert self.numRows() == othermatrix.numRows();
        assert self.numCols() == othermatrix.numCols();
        newmatrix = SparseMatrix(self.numRows(), self.numCols());
        
        for i in range(self.numRows()):
            currentnode = self.rowlist[i];
            while (currentnode is not None):
                newmatrix.setItem([i, currentnode.column], currentnode.value);
                currentnode = currentnode.next;
                
        for j in range(othermatrix.numRows()):
            currentnode = othermatrix.rowlist[j];
            while (currentnode is not None):
                value = newmatrix.getItem([j, currentnode.column]) - currentnode.value;
                newmatrix.setItem([j, currentnode.column], value);
                currentnode = currentnode.next;
        return newmatrix;
    
    
    def transpose(self):
        newmatrix = SparseMatrix(self.numRows(), self.numCols());
        for i in range(self.numRows()):
            currentnode = self.rowlist[i];
            while (currentnode is not None):
                newmatrix.setItem([currentnode.column, i], currentnode.value);
                currentnode = currentnode.next;
        return newmatrix;
    
    def __mul__(self, othermatrix):
        newmatrix = SparseMatrix(self.numRows(), self.numCols());
        for i in range(self.numRows()):
            for j in range(self.numCols()):
                currentnode = self.rowlist[i];
                value = 0;
                while (currentnode is not None):
                    value = value + (currentnode.value * othermatrix.getItem([currentnode.column, j]));
                    currentnode = currentnode.next;
                newmatrix.setItem([i, j], value);
        return newmatrix;
                                     
    def __str__(self):
        s = "[";
        for i in range(self.numRows()):
            for j in range(self.numCols()):
                if j != self.numCols() -1:
                    s += str(self.getItem([i, j])) + "    ";
                else:
                    s += str(self.getItem([i, j]));
            if i != self.numRows() -1:
                s += "\n";
        s += "]";
        return s
                                     
    
    
matrix1 = SparseMatrix(3, 3);
matrix2 = SparseMatrix(3, 3);

#To setup matrices with random values
for i in range(matrix1.numRows()):
    for j in range(matrix1.numCols()):
        matrix1.setItem([i, j], random.randint(0, 10)); 
        matrix2.setItem([i, j], random.randint(0, 10));
        
print("Matrix 1: \n" + str(matrix1));
print("Matrix 2: \n" + str(matrix2));
print("");
matrix3 = matrix1 + matrix2;
matrix4 = matrix1 - matrix2;
print("Matrix 3: \n" + str(matrix3));
print("Matrix 4: \n" + str(matrix4));
print("");
matrix5 = matrix3 * matrix4
print("Matrix5 \n" + str(matrix5));
print("");
matrix6 = matrix5.transpose();
print("Matrix 6: \n" + str(matrix6))

Matrix 1: 
[6    4    3
0    9    5
9    5    9]
Matrix 2: 
[8    2    0
8    8    7
1    1    6]

Matrix 3: 
[14    6    3
8    17    12
10    6    15]
Matrix 4: 
[-2    2    3
-8    1    -2
8    4    3]

Matrix5 
[-52    46    39
-56    81    26
52    86    63]

Matrix 6: 
[-52    -56    52
46    81    86
39    26    63]


In [57]:
class PolyNode:
    def __init__ (self, degree, co):
        self.degree = degree;
        self.coeff = co;
        self.next = None;

class Polynomial:
    def __init__(self, degree = None, co = None):
        if degree is None:
            self.polyhead = None;
        else:
            self.polyhead = PolyNode(degree, co);
        self.tail = self.polyhead;
        
    def addTerm(self, degree, co):
        newpoly = PolyNode(degree, co);
        currentnode = self.polyhead;
        if currentnode is not None:
            if degree == currentnode.degree:
                currentnode.coeff = currentnode.coeff + newpoly.coeff;
                return
            if degree > currentnode.degree:
                newpoly.next = self.polyhead;
                self.polyhead = newpoly;
                return 
            else:
                while (currentnode.next is not None and degree <= currentnode.next.degree):
                    currentnode = currentnode.next;
                    if degree == currentnode.degree:
                        currentnode.coeff = currentnode.coeff + newpoly.coeff;
                        return;
                newpoly.next = currentnode.next;
                currentnode.next = newpoly;
                return;
        else:
            self.polyhead = newpoly;
            
    def degree(self):
        if self.polyhead == None:
            return -1;
        return self.polyhead.degree;
    
    def getItem(self, degree):
        found = False;
        currentnode = self.polyhead;
        if currentnode is not None:
            while (currentnode.next is not None and currentnode.degree != degree):
                currentnode = currentnode.next;
            if degree == currentnode.degree:
                return currentnode.coeff;
            return found;
        else:
            return found;
            
    def evaluate(self, scaler):
        currentnode = self.polyhead;
        if currentnode is not None:
            while (currentnode.next is not None):
                currentnode.coeff *= scaler;
                currentnode = currentnode.next;
            currentnode.coeff *= scaler;
        else:
            raise Exception;
            
    
    def __str__(self):
        s = "[";
        currentnode = self.polyhead;
        if currentnode is not None:
            while (currentnode.next is not None):
                s += str(currentnode.coeff) + "x" + str(currentnode.degree) + " ";
                currentnode = currentnode.next;
            s += str(currentnode.coeff) + "x" + str(currentnode.degree) + "]";
            return s;
        else:
            return None;
        
        
    def __add__(self, poly):
        assert self.degree() != -1 and poly.degree() != -1;
        newpoly = Polynomial();
        
        nodea = self.polyhead;
        nodeb = poly.polyhead;
        
        while (nodea is not None and nodeb is not None):
            if nodea.degree > nodeb.degree:
                degree = nodea.degree;
                coeff = nodea.coeff;
                nodea = nodea.next;
            
            elif nodea.degree < nodeb.degree:
                degree = nodeb.degree;
                coeff = nodeb.coeff;
                nodeb = nodeb.next;
                
            else:
                degree = nodea.degree;
                coeff = nodea.coeff + nodeb.coeff;
                nodea = nodea.next;
                nodeb = nodeb.next;
            
            newpoly.addTerm(degree, coeff);
        nodea = self.polyhead;
        nodeb = poly.polyhead;
        while (nodea is not None):
            if newpoly.getItem(nodea.degree) == 0:
                newpoly.addTerm(nodea.degree, nodea.coeff);
                nodea = nodea.next;
            else:
                nodea = nodea.next;
               
        while (nodeb is not None):
            if newpoly.getItem(nodeb.degree) == 0:
                newpoly.addTerm(nodeb.degree, nodeb.coeff);
            nodeb = nodeb.next
        return newpoly;
    
    def __sub__(self, poly):
        assert self.degree() != -1 and poly.degree() != -1;
        newpoly = Polynomial();
        
        nodea = self.polyhead;
        nodeb = poly.polyhead;
        
        zeroTerms = [];
        while (nodea is not None and nodeb is not None):
            zero = False;
            if nodea.degree > nodeb.degree:
                degree = nodea.degree;
                coeff = nodea.coeff;
                nodea = nodea.next;
            
            elif nodea.degree < nodeb.degree:
                degree = nodeb.degree;
                coeff = nodeb.coeff;
                nodeb = nodeb.next;
                
            else:
                degree = nodea.degree;
                coeff = nodea.coeff - nodeb.coeff;
                nodea = nodea.next;
                nodeb = nodeb.next;
                if coeff == 0:
                    zero = True;
                    zeroTerms.append(degree);
            if not zero:
                newpoly.addTerm(degree, coeff);
        nodea = self.polyhead;
        nodeb = poly.polyhead;
        while (nodea is not None):
            if newpoly.getItem(nodea.degree) == 0 and nodea.degree not in zeroTerms:
                newpoly.addTerm(nodea.degree, nodea.coeff);
                nodea = nodea.next;
            else:
                nodea = nodea.next;
               
        while (nodeb is not None):
            if newpoly.getItem(nodeb.degree) == 0 and nodeb.degree not in zeroTerms:
                newpoly.addTerm(nodeb.degree, nodeb.coeff);
            nodeb = nodeb.next
        return newpoly;
    
    
    def __mul__(self, poly):
        assert self.degree() != -1 and poly.degree() != -1;
        newpoly = Polynomial();
        
        nodea = self.polyhead;
        
        while (nodea is not None):
            nodeb = poly.polyhead;
            while (nodeb is not None):
                degree = nodea.degree + nodeb.degree;
                coeff = nodea.coeff * nodeb.coeff;
                nodeb = nodeb.next;
                newpoly.addTerm(degree, coeff);
            nodea = nodea.next;
        return newpoly; 

neweq = Polynomial(3, 3);
neweq.addTerm(2, 4)
neweq.addTerm(5, 12)
neweq.addTerm(5, 12)
neweq.addTerm(2, 4)
neweq.addTerm(0, 5)
neweq.addTerm(3, 5)

print("neweq: ", neweq)
second = Polynomial();
second.addTerm(5, 9);
second.addTerm(4, 12);
second.addTerm(3, 3);
second.addTerm(2, 8);
second.addTerm(1, 9);
second.addTerm(6, 9);
second.addTerm(0, 12)
print("second eq: ", second)
third = neweq + second
print("Third eq: ", third)
fourth = neweq - second;
print("fourth eq: ", fourth)
fifth = neweq * second;
print("Fifth: ", fifth)

neweq:  [24x5 8x3 8x2 5x0]
second eq:  [9x6 9x5 12x4 3x3 8x2 9x1 12x0]
Third eq:  [9x6 33x5 12x4 11x3 16x2 9x1 17x0]
fourth eq:  [9x6 15x5 12x4 5x3 9x1 -7x0]
Fifth:  [216x11 216x10 360x9 216x8 360x7 381x6 421x5 196x4 183x3 136x2 45x1 60x0]
